# Importing modules

In [6]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

In [26]:
sale = pd.read_csv('data/EXTR_RPSale.csv')
sale.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,2687551,138860,110,08/21/2014,245000,20140828001436,,,,,...,3,6,3,N,N,N,N,1,8,
1,1235111,664885,40,07/09/1991,0,199203161090,071,001,664885,C,...,3,0,26,N,N,N,N,18,3,11
2,2704079,423943,50,10/11/2014,0,20141205000558,,,,,...,3,6,15,N,N,N,N,18,8,18 31 51
3,2584094,403700,715,01/04/2013,0,20130110000910,,,,,...,3,6,15,N,N,N,N,11,8,18 31 38
4,1056831,951120,900,04/20/1989,85000,198904260448,117,053,951120,P,...,3,0,2,N,N,N,N,1,9,49


In [28]:
parcel = pd.read_csv('data/EXTR_Parcel.csv', encoding='latin-1')
parcel.head()

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,259750,840,,FOREST RIDGE ADD,22,4,5,21,29,NE,...,N,N,N,N,N,N,N,N,N,N
1,221221,480,,EASTRIDGE DIV NO. 02,48 & 49,,5,22,17,NE,...,N,N,N,N,N,N,N,N,N,N
2,57900,1110,,BAY VIEW ADD TO SALMON BAY,11-12-13,9,3,26,35,NE,...,N,N,N,N,N,N,N,N,N,N
3,611540,83,,NORMANDY PARK FIVE-ACRE TRS,POR 4,,4,22,6,NE,...,N,N,N,N,N,N,N,N,N,N
4,605611,800,,NEW RAINIER VISTA II,1D,40,4,24,16,SE,...,N,N,N,N,N,N,N,N,N,N


In [29]:
res_bldg = pd.read_csv('data/EXTR_ResBldg.csv')
res_bldg.head()    

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,10455,780,1,1,32708 19TH PL SW 98023,32708,,,19TH,PL,...,1,0,0,1989,0,0,0,0,3,0
1,10455,810,1,1,32616 19TH PL SW 98023,32616,,,19TH,PL,...,1,0,0,1989,0,0,0,0,4,0
2,10455,860,1,1,32619 18TH PL SW 98023,32619,,,18TH,PL,...,1,0,0,1989,0,0,0,0,3,0
3,10456,410,1,1,32603 20TH AVE SW 98023,32603,,,20TH,AVE,...,0,0,0,1990,0,0,0,0,3,0
4,10457,810,1,1,33010 20TH AVE SW 98023,33010,,,20TH,AVE,...,1,0,0,1992,0,0,0,0,3,0


In [33]:
'''
PIN is the unique identifier, which combines the major and minor fields. Major should be 6 digits, minor should be 4
digits long. Add leading zeros if they are not the specified length.
'''
sale['Major'] = sale['Major'].apply(lambda x: str(x).zfill(6))
sale['Minor'] = sale['Minor'].apply(lambda x: str(x).zfill(4))
sale.head()      

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,2687551,138860,0110,08/21/2014,245000,20140828001436,,,,,...,3,6,3,N,N,N,N,1,8,
1,1235111,664885,0040,07/09/1991,0,199203161090,071,001,664885,C,...,3,0,26,N,N,N,N,18,3,11
2,2704079,423943,0050,10/11/2014,0,20141205000558,,,,,...,3,6,15,N,N,N,N,18,8,18 31 51
3,2584094,403700,0715,01/04/2013,0,20130110000910,,,,,...,3,6,15,N,N,N,N,11,8,18 31 38
4,1056831,951120,0900,04/20/1989,85000,198904260448,117,053,951120,P,...,3,0,2,N,N,N,N,1,9,49


In [34]:
parcel['Major'] = parcel['Major'].apply(lambda x: str(x).zfill(6))
parcel['Minor'] = parcel['Minor'].apply(lambda x: str(x).zfill(4))
parcel.head() 

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,259750,0840,,FOREST RIDGE ADD,22,4,5,21,29,NE,...,N,N,N,N,N,N,N,N,N,N
1,221221,0480,,EASTRIDGE DIV NO. 02,48 & 49,,5,22,17,NE,...,N,N,N,N,N,N,N,N,N,N
2,057900,1110,,BAY VIEW ADD TO SALMON BAY,11-12-13,9,3,26,35,NE,...,N,N,N,N,N,N,N,N,N,N
3,611540,0083,,NORMANDY PARK FIVE-ACRE TRS,POR 4,,4,22,6,NE,...,N,N,N,N,N,N,N,N,N,N
4,605611,0800,,NEW RAINIER VISTA II,1D,40,4,24,16,SE,...,N,N,N,N,N,N,N,N,N,N


In [35]:
res_bldg['Major'] = res_bldg['Major'].apply(lambda x: str(x).zfill(6))
res_bldg['Minor'] = res_bldg['Minor'].apply(lambda x: str(x).zfill(4))
res_bldg.head() 

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,010455,0780,1,1,32708 19TH PL SW 98023,32708,,,19TH,PL,...,1,0,0,1989,0,0,0,0,3,0
1,010455,0810,1,1,32616 19TH PL SW 98023,32616,,,19TH,PL,...,1,0,0,1989,0,0,0,0,4,0
2,010455,0860,1,1,32619 18TH PL SW 98023,32619,,,18TH,PL,...,1,0,0,1989,0,0,0,0,3,0
3,010456,0410,1,1,32603 20TH AVE SW 98023,32603,,,20TH,AVE,...,0,0,0,1990,0,0,0,0,3,0
4,010457,0810,1,1,33010 20TH AVE SW 98023,33010,,,20TH,AVE,...,1,0,0,1992,0,0,0,0,3,0


In [36]:
sale['PIN'] = sale['Major'] + sale['Minor']
parcel['PIN'] = parcel['Major'] + parcel['Minor']
res_bldg['PIN'] = res_bldg['Major'] + res_bldg['Minor']

In [37]:
sale.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,PIN
0,2687551,138860,0110,08/21/2014,245000,20140828001436,,,,,...,6,3,N,N,N,N,1,8,,1388600110
1,1235111,664885,0040,07/09/1991,0,199203161090,071,001,664885,C,...,0,26,N,N,N,N,18,3,11,6648850040
2,2704079,423943,0050,10/11/2014,0,20141205000558,,,,,...,6,15,N,N,N,N,18,8,18 31 51,4239430050
3,2584094,403700,0715,01/04/2013,0,20130110000910,,,,,...,6,15,N,N,N,N,11,8,18 31 38,4037000715
4,1056831,951120,0900,04/20/1989,85000,198904260448,117,053,951120,P,...,0,2,N,N,N,N,1,9,49,9511200900
